In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import numpy as np

In [28]:
nls_experiment = pd.read_csv("C:\\Users\\jyoti\\Simon MSBA\\Fall 24\\Fall A\\Analytics Design and Application\\Case 5\\nls_experiment.csv")
nls_experiment.head(10)

,Employee_ID,Office_ID,Course,Intake_Proficiency_Score,Intake_Applications_Score,Outcome_Proficiency_Score,Outcome_Applications_Score
0,900268,1,103,0.235537,0.229980,0.523067,0.675717
1,900275,6,103,0.238583,0.206941,0.609806,0.654843
2,900305,10,103,0.303458,0.334426,0.726753,0.693479
3,900313,12,103,0.312748,0.425477,0.876779,0.653169
4,900335,12,103,0.369744,0.198643,0.655737,0.596176
5,900338,6,103,0.185036,0.340688,0.642293,0.680473
6,900411,10,103,0.303497,0.369944,0.760330,0.673719
7,900507,3,103,0.250395,0.243314,0.470421,0.538566
8,900703,10,103,0.317634,0.395539,0.737099,0.735404
9,900861,1,103,0.190878,0.221601,0.740856,0.623202


In [29]:
# Identifying unique Office_IDs in the dataset to refine the mapping
unique_offices = nls_experiment['Office_ID'].unique()
unique_offices


array([ 1,  6, 10, 12,  3,  8], dtype=int64)

In [30]:
office_mapping = {
    1: ('Miami', 'A'),
    6: ('Houston', 'A'),
    10: ('Detroit', 'B'),
    12: ('Denver', 'B'),
    3: ('New York', 'Current'),  # Add New York for Office ID 3
    8: ('Los Angeles', 'Current')  # Add Los Angeles for Office ID 8
}
nls_experiment['Location'] = nls_experiment['Office_ID'].map(lambda x: office_mapping[x][0])
nls_experiment['Training_Program'] = nls_experiment['Office_ID'].map(lambda x: office_mapping[x][1])
summary = nls_experiment.groupby('Location').agg({
    'Employee_ID': 'count',
    'Training_Program': 'first'
}).rename(columns={'Employee_ID': 'Number of Entries'}).reset_index()

print(summary)


      Location  Number of Entries Training_Program
0       Denver                109                B
1      Detroit                101                B
2      Houston                183                A
3  Los Angeles                 77          Current
4        Miami                 69                A
5     New York                 54          Current


In [31]:
summary.head()

,Location,Number of Entries,Training_Program
0,Denver,109,B
1,Detroit,101,B
2,Houston,183,A
3,Los Angeles,77,Current
4,Miami,69,A


In [32]:
nls_experiment.head()

,Employee_ID,Office_ID,Course,Intake_Proficiency_Score,Intake_Applications_Score,Outcome_Proficiency_Score,Outcome_Applications_Score,Location,Training_Program
0,900268,1,103,0.235537,0.229980,0.523067,0.675717,Miami,A
1,900275,6,103,0.238583,0.206941,0.609806,0.654843,Houston,A
2,900305,10,103,0.303458,0.334426,0.726753,0.693479,Detroit,B
3,900313,12,103,0.312748,0.425477,0.876779,0.653169,Denver,B
4,900335,12,103,0.369744,0.198643,0.655737,0.596176,Denver,B


In [33]:
#reordering to ensure Office_ID locations are right next to each other

In [34]:
# Reorder columns so 'Office_ID' and 'Location' are next to each other
columns_order = ['Employee_ID', 'Office_ID', 'Location', 'Training_Program',
                 'Course', 'Intake_Proficiency_Score', 'Intake_Applications_Score',
                 'Outcome_Proficiency_Score', 'Outcome_Applications_Score']

# Reordering the columns in the DataFrame
nls_experiment = nls_experiment[columns_order]

# Display the first few rows to verify
nls_experiment.head()


,Employee_ID,Office_ID,Location,Training_Program,Course,Intake_Proficiency_Score,Intake_Applications_Score,Outcome_Proficiency_Score,Outcome_Applications_Score
0,900268,1,Miami,A,103,0.235537,0.229980,0.523067,0.675717
1,900275,6,Houston,A,103,0.238583,0.206941,0.609806,0.654843
2,900305,10,Detroit,B,103,0.303458,0.334426,0.726753,0.693479
3,900313,12,Denver,B,103,0.312748,0.425477,0.876779,0.653169
4,900335,12,Denver,B,103,0.369744,0.198643,0.655737,0.596176


In [35]:
nls_experiment_exluded = nls_experiment.drop(['Office_ID'], axis =1)
nls_experiment_exluded = nls_experiment
nls_experiment.head()

,Employee_ID,Office_ID,Location,Training_Program,Course,Intake_Proficiency_Score,Intake_Applications_Score,Outcome_Proficiency_Score,Outcome_Applications_Score
0,900268,1,Miami,A,103,0.235537,0.229980,0.523067,0.675717
1,900275,6,Houston,A,103,0.238583,0.206941,0.609806,0.654843
2,900305,10,Detroit,B,103,0.303458,0.334426,0.726753,0.693479
3,900313,12,Denver,B,103,0.312748,0.425477,0.876779,0.653169
4,900335,12,Denver,B,103,0.369744,0.198643,0.655737,0.596176


In [36]:
# Step 1: Calculate improvement scores
nls_experiment['Proficiency_Improvement'] = (
    nls_experiment['Outcome_Proficiency_Score'] - nls_experiment['Intake_Proficiency_Score']
)
nls_experiment['Applications_Improvement'] = (
    nls_experiment['Outcome_Applications_Score'] - nls_experiment['Intake_Applications_Score']
)

# Create dummy variables
training_dummies = pd.get_dummies(nls_experiment['Training_Program'], prefix='Training_Program')
nls_experiment = pd.concat([nls_experiment, training_dummies], axis=1)

# Set up features and targets
X = nls_experiment[['Training_Program_B', 'Training_Program_Current']]  # A is baseline
y_proficiency = nls_experiment['Proficiency_Improvement']
y_applications = nls_experiment['Applications_Improvement']
X = sm.add_constant(X)


In [38]:
#check for data types 
print(nls_experiment[['Proficiency_Improvement', 'Applications_Improvement', 'Training_Program_B', 'Training_Program_Current']].dtypes)


Proficiency_Improvement     float64
Applications_Improvement    float64
Training_Program_B             bool
Training_Program_Current       bool
dtype: object


In [39]:
#convert Training_Program_B and Training Program current to integes

nls_experiment['Training_Program_B'] = nls_experiment['Training_Program_B'].astype(int)
nls_experiment['Training_Program_Current'] = nls_experiment['Training_Program_Current'].astype(int)


In [40]:
#Proficieny 

X = nls_experiment[['Training_Program_B', 'Training_Program_Current']]
y_proficiency = nls_experiment['Proficiency_Improvement']


# Add a constant to the independent variables
X = sm.add_constant(X)

# Build the linear regression models
model_proficiency = sm.OLS(y_proficiency, X).fit()
model_applications = sm.OLS(y_applications, X).fit()

# Display the summaries of the models
print(model_proficiency.summary())
print(model_applications.summary())


                               OLS Regression Results                              
Dep. Variable:     Proficiency_Improvement   R-squared:                       0.069
Model:                                 OLS   Adj. R-squared:                  0.066
Method:                      Least Squares   F-statistic:                     21.78
Date:                     Sun, 20 Apr 2025   Prob (F-statistic):           7.50e-10
Time:                             20:37:49   Log-Likelihood:                 590.60
No. Observations:                      593   AIC:                            -1175.
Df Residuals:                          590   BIC:                            -1162.
Df Model:                                2                                         
Covariance Type:                 nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------

In [41]:
#Application Improvement
y_applications = nls_experiment['Applications_Improvement']
X = sm.add_constant(X)
model_applications = sm.OLS(y_applications, X).fit()
print(model_applications.summary())

                               OLS Regression Results                               
Dep. Variable:     Applications_Improvement   R-squared:                       0.205
Model:                                  OLS   Adj. R-squared:                  0.202
Method:                       Least Squares   F-statistic:                     76.02
Date:                      Sun, 20 Apr 2025   Prob (F-statistic):           4.21e-30
Time:                              20:37:49   Log-Likelihood:                 671.93
No. Observations:                       593   AIC:                            -1338.
Df Residuals:                           590   BIC:                            -1325.
Df Model:                                 2                                         
Covariance Type:                  nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------